## XX. Fastchat-T5 3B v1.0 PEFT LoRA fine-tunning

In [ ]:
# Definir o dispositivo CUDA
device = torch.device('cuda:3')
torch.cuda.set_device(device)

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3'

In [2]:
os.environ.get('CUDA_VISIBLE_DEVICES')

'1,2,3'

In [3]:
import os
for key, value in os.environ.items():
    print(key, '=', value)

CUDA_PATH = /usr/local/cuda-12.3
CUDA_INC_PATH = /usr/local/cuda-12.3/targets/x86_64-linux
XDG_SESSION_ID = 13590
HOSTNAME = dxl1niagp001
SSL_CERT_FILE = /etc/pki/ca-trust/extracted/pem/tls-ca-bundle.pem
SHELL = /bin/bash
HISTSIZE = 1000
SSH_CLIENT = 172.20.160.54 43958 22
LIBRARY_PATH = /usr/local/cuda-12.3/lib64:/usr/local/cuda-12.3/lib64:
CONDA_SHLVL = 1
CONDA_PROMPT_MODIFIER = (base) 
NO_PROXY = localhost,127.0.0.1,bb.com.br,localaddress,.localdomain.com
CUDA_TOOLKIT_ROOT_DIR = /usr/local/cuda-12.3
CUDA_HOME = /usr/local/cuda-12.3
USER = c1331397
LD_LIBRARY_PATH = /usr/local/cuda-11.5/lib64:/usr/local/cuda-12.3/lib:/usr/local/cuda-12.3/lib64:/usr/local/cuda-11.5/lib64:/usr/local/cuda-12.3/lib:/usr/local/cuda-12.3/lib64:/usr/local/cuda/lib64:/usr/lib64/nvidia/gridd
CONDA_EXE = /var/ontologia/Servicos/ACS_LLMS/server/anaconda3/bin/conda
CPATH = /usr/local/cuda-12.3/include:/usr/local/cuda-12.3/include:
CUDA_TOOLKIT_ROOT = /usr/local/cuda-12.3
_CE_CONDA = 
VSCODE_AGENT_FOLDER = /var/o

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration # ou AutoModelForSeq2SeqLM
from transformers import GenerationConfig, TrainingArguments, Trainer
import torch
from datasets import Dataset
import pandas as pd
import numpy as np


model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/fastchat-t5-3b-v1.0'

original_model = T5ForConditionalGeneration.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = T5Tokenizer.from_pretrained(model_path)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to German: "
    },
    "translation_en_to_fr": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to French: "
    },
    "translation_en_to_ro": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to Romanian: "
    }
  },
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.36.0.dev",
  "use_cache": true,
  "vocab_size": 32128
}

In [2]:
# Função para puxar o número de parâmetros/pesos do modelo (totais) e os que requerem gradiente (treináveis)
# Razão número de pesos treináveis / totais

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


In [ ]:
print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 2849683456
all model parameters: 2849683456
percentage of trainable model parameters: 100.00%


### 1 - Train Dataset

In [25]:
# Processar o arquivo e criar um DataFrame
data = []
with open('dataset_sintetico.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if line.startswith('Texto:'):
            text = line[len('Texto: '):].strip()
        elif line.startswith('Intenção:'):
            intention = line[len('Intenção: '):].strip()
            data.append({'text': text, 'intention': intention})

df = pd.DataFrame(data)

In [ ]:
df

,text,intention
0,Quero cancelar minha compra recente.,Ajuda com entendimento de fatura
1,Como faço para acompanhar minha entrega?,Problemas com aplicativo móvel
2,Preciso de ajuda para entender minha fatura.,Cancelamento de compra
3,Quero informações sobre a garantia do produto.,Alteração de endereço de entrega
4,Como posso atualizar minhas informações de pag...,Horários de funcionamento do suporte
...,...,...
439,Como faço para alterar o endereço cadastrado n...,Alteração de endereço cadastrado
440,Posso ter uma extensão do período de teste do ...,Solicitação de extensão de período de teste
441,Quais são as opções de suporte para o fim de s...,Opções de suporte no fim de semana
442,Como posso enviar um feedback sobre o atendime...,Envio de feedback sobre atendimento


In [26]:
# Função de tokenização

def tokenize_function(dataset):
    input_ids = tokenizer(dataset['text'], padding="max_length", truncation=True, return_tensors="pt").input_ids
    labels = tokenizer(dataset['intention'], padding="max_length", truncation=True, return_tensors="pt").input_ids
    return {'input_ids': input_ids.squeeze(), 'labels': labels.squeeze()}

In [27]:
# Aplicar a tokenização ao DataFrame
tokenized_data = df.apply(tokenize_function, axis=1)

# Converter o resultado em listas
input_ids = list(tokenized_data.apply(lambda x: x['input_ids']))
labels = list(tokenized_data.apply(lambda x: x['labels']))


In [28]:
# Criar um Dataset
tokenized_train_dataset = Dataset.from_dict({'input_ids': input_ids, 'labels': labels})

In [18]:
print(tokenized_train_dataset)

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 444
})


<a name='1'></a>
### 2 - Setup the PEFT/LoRA model for Fine-Tuning

You need to set up the PEFT/LoRA model for fine-tuning with a new layer/parameter adapter. Using PEFT/LoRA, you are freezing the underlying LLM and only training the adapter. Have a look at the LoRA configuration below. Note the rank (`r`) hyper-parameter, which defines the rank/dimension of the adapter to be trained.

In [29]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

It is possible to pull out the number of model parameters and find out how many of them are trainable. The following function can be used to do that, at this stage, you do not need to go into details of it. 

Add LoRA adapter layers/parameters to the original LLM to be trained.

In [30]:
peft_model = get_peft_model(original_model, 
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 4718592
all model parameters: 2854402048
percentage of trainable model parameters: 0.17%


<a name='3.2'></a>
### 3.2 - Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [31]:
import time

output_dir = f'/var/ontologia/Servicos/ACS_LLMS/server/models/peft-fastchat-t5-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-4, # Higher learning rate than full fine-tuning
    num_train_epochs=5,
    logging_steps=10,
    max_steps=60   
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train_dataset,
)

""" Base learning rate: 1e-4
A learning rate of 1e-4 has become the standard when fine-tuning LLMs with LoRA. 
Although we occasionally encountered training loss instabilities, reducing the learning rate to lower values like 3e-5 proved effective in stabilizing 
the process — more on this will follow. """

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


' Base learning rate: 1e-4\nA learning rate of 1e-4 has become the standard when fine-tuning LLMs with LoRA. \nAlthough we occasionally encountered training loss instabilities, reducing the learning rate to lower values like 3e-5 proved effective in stabilizing \nthe process — more on this will follow. '

Now everything is ready to train the PEFT adapter and save the model.

In [32]:
peft_trainer.train()

peft_model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-fastchat-t5-3b-3'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
1,18.875000


('/var/ontologia/Servicos/ACS_LLMS/server/models/peft-fastchat-t5-3b-3/tokenizer_config.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-fastchat-t5-3b-3/special_tokens_map.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-fastchat-t5-3b-3/spiece.model',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-fastchat-t5-3b-3/added_tokens.json')

### Sucesso. Modelo PEFT LoRA adaptado criado no diretório

<hr>

# Inferência com o modelo PEFT LoRA

In [33]:
from peft import PeftModel, PeftConfig
from transformers import T5Tokenizer, T5ForConditionalGeneration  # ou AutoModelForSeq2SeqLM
import torch


peft_model_base = T5ForConditionalGeneration.from_pretrained('/var/ontologia/Servicos/ACS_LLMS/server/models/fastchat-t5-3b-v1.0_copy', torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = T5Tokenizer.from_pretrained('/var/ontologia/Servicos/ACS_LLMS/server/models/fastchat-t5-3b-v1.0_copy')

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-fastchat-t5-3b-3', 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False, 
                                       device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The number of trainable parameters will be `0` due to `is_trainable=False` setting:

In [16]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 2868631552
percentage of trainable model parameters: 0.00%


<hr>

############################################  TESTES DE PROMPT  ############################################

In [34]:
################# Classificação de Intenções #################

context = """


"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço   apresentada   é   de   perguntar   sobre   como   atualizar   o   carto   de   crédito. 



In [17]:
################# Classificação de Intenções #################

context = """


"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço   apresentada   é   de   perguntar   sobre   como   atualizar   o   carto   de   crédito. 



In [ ]:
################# Classificação de Intenções #################

context = """


"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço   apresentada   é   de   perguntar   sobre   como   atualizar   o   carto   de   crédito. 



In [16]:
################# Classificação de Intenções #################

context = """


"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João 

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


<pad> Intenço:  Perguntar  sobre  como  atualizar  um  carto  de  crédito



In [36]:
################# Classificação de Intenções #################

context = """
Texto:
"Podem me informar como posso participar do programa de fidelidade e quais são os benefícios oferecidos?"
Intenção: Solicitar informações sobre programa de fidelidade.

Texto:
"Estou com dúvidas sobre os ingredientes utilizados nos produtos da linha de cuidados com a pele. Vocês podem fornecer mais detalhes?"
Intenção: Inquirir sobre informações de produto.

"""

question = """
Denys, it is actually super helpful that you chose to do this live with us instead of only recording the "happy path". Thanks for sharing. 
I would also manipulate the max_tokens variable or do something like "only use one character in your response".

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço:   Criar   uma   recomendaço   ou   sugerir   uma   melhor   abordagem   para   o   objetivo   especfico   do   texto. 



In [37]:
################# Classificação de Intenções #################

context = """
Texto:
"Podem me informar como posso participar do programa de fidelidade e quais são os benefícios oferecidos?"
Intenção: Solicitar informações sobre programa de fidelidade.

Texto:
"Estou com dúvidas sobre os ingredientes utilizados nos produtos da linha de cuidados com a pele. Vocês podem fornecer mais detalhes?"
Intenção: Inquirir sobre informações de produto.

"""

question = """
Our paper exploring zero and few-shot techniques for intent classification using LLMs was presented in the ACL conference (a top-tier conference in NLP areas) last month. 


Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço:   Explorar   técnicas   de   intento   classificaço   utilizando   LLMs. 



In [35]:
################# Reconhecimento e Extração de Entidades #################


context = """
Texto:
"A Microsoft, fundada por Bill Gates e Paul Allen, está sediada em Redmond, Washington."

Análise:
{
  "pessoa": ["Bill Gates", "Paul Allen"],
  "lugar": ["Redmond", "Washington"],
  "organização": ["Microsoft"]
}

"""

question = """
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.

Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

` ` "entities":   ["Tim   Cook",   "CEO",   "Apple",   "Steve   Jobs   Theatre",   "Cupertino",   "California",   "iPhone   15",   "várias   cores",   "processador",   "bateria"] } ` `


<hr>

############################################  CLASSIFICAÇÃO DE INTENÇÕES  ############################################

In [4]:
# Configurar o otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Ler o dataset sintético e usá-lo como contexto
with open('dataset_sintetico.txt', 'r') as file:
    context = file.read()

# Sua pergunta específica
question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João

Classifique a intenção presente no texto.
"""

# Combinar contexto e pergunta para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"



# Tokenização do prompt
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Preparar o otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Criar decoder_input_ids
decoder_input_ids = inputs.input_ids[:, :-1]
labels = inputs.input_ids[:, 1:]  # Labels são deslocados para a direita

# Forward pass
model.train()
outputs = model(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask, decoder_input_ids=decoder_input_ids, labels=labels)

# Calcular a perda
loss = outputs.loss
print(f"Loss: {loss.item()}")  # Exibir o resultado da função de perda

# Backward pass
loss.backward() # "backpropagation" na rede "feed forward"

# Atualizar os pesos do modelo
optimizer.step()

# Limpar os gradientes
optimizer.zero_grad()


Loss: 3.019024610519409


In [16]:
print(loss)

tensor(0.2733, grad_fn=<ToCopyBackward0>)


In [17]:
print(f"Loss: {loss.item()}")  # Exibir a perda

Loss: 0.27331289649009705


In [10]:
# Após o treinamento, salvar o modelo
model.save_pretrained('/var/ontologia/Servicos/ACS_LLMS/server/models/fastchat-t5-3b-v1.0_fine-tunned')

# Salvar o tokenizador, caso ele tenha sido modificado
tokenizer.save_pretrained('/var/ontologia/Servicos/ACS_LLMS/server/models/fastchat-t5-3b-v1.0_fine-tunned')


('/var/ontologia/Servicos/ACS_LLMS/server/models/fastchat-t5-3b-v1.0_fine-tunned/tokenizer_config.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/fastchat-t5-3b-v1.0_fine-tunned/special_tokens_map.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/fastchat-t5-3b-v1.0_fine-tunned/spiece.model',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/fastchat-t5-3b-v1.0_fine-tunned/added_tokens.json')

<hr>

############################################  TESTES DE PROMPT  ############################################

In [1]:
##### Carregar novo modelo ajustado

from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
import torch

model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/fastchat-t5-3b-v1.0_fine-tunned'
model = T5ForConditionalGeneration.from_pretrained(model_path, device_map="auto")
tokenizer = T5Tokenizer.from_pretrained(model_path) 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
################# Classificação de Intenções #################

context = """


"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço   apresentada   é   de   perguntar   sobre   como   atualizar   o   carto   de   crédito. 



In [17]:
################# Classificação de Intenções #################

context = """
Texto:
"Estou tendo problemas para fazer login na minha conta, e já tentei redefinir minha senha várias vezes sem sucesso. O que mais eu posso fazer para resolver isso?"
Intenção: Solicitar assistência para o acesso à conta.

Texto:
"Acabei de receber o produto que comprei de vocês, e estou muito satisfeito com a qualidade. Quero agradecer e dizer que continuarei sendo um cliente fiel."
Intenção: Expressar satisfação e gratidão.

"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço:   Solicitar   assistência   para   adicionar   um   novo   carto   de   crédito. 



In [23]:
################# Classificação de Intenções #################

context = """

"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço   objetiva:   ajuda   a   atualizar   um   carto   de   crédito 



In [24]:
################# Classificação de Intenções #################

context = """
Texto:
"Podem me informar como posso participar do programa de fidelidade e quais são os benefícios oferecidos?"
Intenção: Solicitar informações sobre programa de fidelidade.

Texto:
"Estou com dúvidas sobre os ingredientes utilizados nos produtos da linha de cuidados com a pele. Vocês podem fornecer mais detalhes?"
Intenção: Inquirir sobre informações de produto.

"""

question = """
Denys, it is actually super helpful that you chose to do this live with us instead of only recording the "happy path". Thanks for sharing. 
I would also manipulate the max_tokens variable or do something like "only use one character in your response".

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço:   Criar   uma   recomendaço   ou   sugerir   uma   melhoria   para   o   processo   de   conversaço   em   gerenciamento   de   texto. 



In [26]:
################# Classificação de Intenções #################

context = """
Texto:
"Podem me informar como posso participar do programa de fidelidade e quais são os benefícios oferecidos?"
Intenção: Solicitar informações sobre programa de fidelidade.

Texto:
"Estou com dúvidas sobre os ingredientes utilizados nos produtos da linha de cuidados com a pele. Vocês podem fornecer mais detalhes?"
Intenção: Inquirir sobre informações de produto.

"""

question = """
Our paper exploring zero and few-shot techniques for intent classification using LLMs was presented in the ACL conference (a top-tier conference in NLP areas) last month. 


Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço:   Explorar   técnicas   de   intento   classificaço   utilizando   LLMs. 



<hr>

############################################  COMPREENSÃO E GERAÇÃO DE TEXTO  ############################################
############################################  AVALIAÇÃO DA SEMÂNTICA E SINTAXE EM PT-BR  ############################################

In [27]:
################# Question Answering: Compreensão de texto e xtração de conteúdo com Contexto de Domínio #################

# Contexto e questão
context = """
Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investidores que buscam mais rentabilidade do que a poupança e estão dispostos a abrir mão da liquidez diária. 
- **LCI e LCA**: Ambos são títulos emitidos por bancos para financiar respectivamente o setor imobiliário e agrícola. São ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia, como imobiliário ou agrícola. 
- **Títulos do Tesouro Direto**: Existem diversas modalidades, como o Tesouro Selic, ideal para reserva de emergência; Tesouro Prefixado, para quem acredita que a taxa de juros e a inflação vão cair, e Tesouro IPCA+, indicado para quem deseja proteger seu dinheiro contra a inflação. Recomendados para quem busca diversificar sua carteira de investimentos com opções seguras. São títulos públicos emitidos pelo governo federal e são considerados os investimentos mais seguros do país devido à garantia do governo federal. 
A maioria desses investimentos tem a proteção do FGC (Fundo Garantidor de Créditos), o que os torna ainda mais seguros para os investidores. Além disso, o Banco do Brasil, uma das principais instituições financeiras do país, oferece todas essas opções de investimento com diferentes vantagens e condições, atendendo a diversos perfis de investidores, desde os mais conservadores até os mais arrojados.

Pergunta: Com base nos seus conhecimentos e no texto, o que têm em comum LCI e LCA?
Resposta: Ambos são títulos emitidos por bancos para financiar o setor imobiliário e agrícola, ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia.
"""



question = """
Com base nos seus conhecimentos e no texto apresentado sobre investimentos em reda fixa, explique qual é o investimento mais adequado para alguém que busca proteção contra a inflação.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenizar o texto
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

Resposta:   O   Tesouro   IPCA+   é   o   investimento   mais   adequado   para   alguém   que   busca   proteço   contra   a   inflaço.   É   adequado   para   investidores   que   buscam   diversificar   sua   carteira   de   investimentos   com   opçes   seguras   e   com   a   proteço   do   FGC   (Fundo   Garantidor   de   Créditos).   O   Tesouro   IPCA+   tem   a   taxa   definida   com   base   no   tempo   de   permanência   da   aplicaço,   o   que   os   torna   ainda   mais   seguros   para   os   investidores.   Além   disso,   o   Te


Resposta:   O   Tesouro   IPCA+   é   o   investimento   mais   adequado   para   alguém   que   busca   proteço   contra   a   inflaço.   É   adequado   para   investidores   que   buscam   diversificar   sua   carteira   de   investimentos   com   opçes   seguras   e   com   a   proteço   do   FGC   (Fundo   Garantidor   de   Créditos).   O   Tesouro   IPCA+   tem   a   taxa   definida   com   base   no   tempo   de   permanência   da   aplicaço,   o   que   os   torna   ainda   mais   seguros   para   os   investidores.   Além   disso,   o   Te







In [28]:
################# Question Answering com Contexto de Domínio #################

# Contexto e questão
context = """
Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investidores que buscam mais rentabilidade do que a poupança e estão dispostos a abrir mão da liquidez diária. 
- **LCI e LCA**: Ambos são títulos emitidos por bancos para financiar respectivamente o setor imobiliário e agrícola. São ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia, como imobiliário ou agrícola. 
- **Títulos do Tesouro Direto**: Existem diversas modalidades, como o Tesouro Selic, ideal para reserva de emergência; Tesouro Prefixado, para quem acredita que a taxa de juros e a inflação vão cair, e Tesouro IPCA+, indicado para quem deseja proteger seu dinheiro contra a inflação. Recomendados para quem busca diversificar sua carteira de investimentos com opções seguras. São títulos públicos emitidos pelo governo federal e são considerados os investimentos mais seguros do país devido à garantia do governo federal. 
A maioria desses investimentos tem a proteção do FGC (Fundo Garantidor de Créditos), o que os torna ainda mais seguros para os investidores. Além disso, o Banco do Brasil, uma das principais instituições financeiras do país, oferece todas essas opções de investimento com diferentes vantagens e condições, atendendo a diversos perfis de investidores, desde os mais conservadores até os mais arrojados.

Pergunta: Com base nos seus conhecimentos e no texto, o que têm em comum LCI e LCA?
Resposta: Ambos são títulos emitidos por bancos para financiar o setor imobiliário e agrícola, ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia.
"""

question = """model_directory
Com base nos seus conhecimentos e no texto apresentado sobre investimentos em reda fixa, explique o que é Tesouro Selic (em português).
"""


# Combinaçãanswer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)o do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

  Tesouro   Selic   é   um   ttulo   de   dvida   pblico   que   é   apresentado   pelo   Banco   do   Brasil   e   é   adequado   para   reserva   de   emergência.   É   adequado   para   investidores   que   buscam   abrir   mo   da   liquidez   diária   e   esto   dispostos   a   abrir   mo   da   reserva   de   emergência   em   caso   de   necessidade.   O   Tesouro   Selic   tem   uma   taxa   de   juros   de   5%   e   é   adequado   para   investidores   que   buscam   proteger   seu   dinheiro   contra   a  


  Tesouro   Selic   é   um   ttulo   de   dvida   pblico   que   é   apresentado   pelo   Banco   do   Brasil   e   é   adequado   para   reserva   de   emergência.   É   adequado   para   investidores   que   buscam   abrir   mo   da   liquidez   diária   e   esto   dispostos   a   abrir   mo   da   reserva   de   emergência   em   caso   de   necessidade.   O   Tesouro   Selic   tem   uma   taxa   de   juros   de   5%   e   é   adequado   para   investidores   que   buscam   proteger   seu   dinheiro   contra   a  


In [29]:
################# Compreensão e Geração de Texto #################

context = """
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Pergunta: Por que a Grande Barreira de Corais pode ser considerada importante para a biodiversidade marinha?
Resposta: A Grande Barreira de Corais é importante para a biodiversidade marinha devido à sua grande variedade de vida marinha, incluindo um número significativo de espécies de peixes e corais, o que indica um ecossistema rico e diversificado.

Pergunta: Como a localização da Grande Barreira de Corais afeta sua biodiversidade?
Resposta: Sua localização no Mar de Coral, que possui condições ideais como águas quentes e claras, favorece a biodiversidade, permitindo que um grande número de espécies de corais e peixes prospere.
"""

question = """
Dada a descrição do contexto acima:
Pergunta: Explique como a diversidade de espécies na Grande Barreira de Corais contribui para a sua importância como um ecossistema marinho.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

A   diversidade   de   espécies   na   Grande   Barreira   de   Corais   contribui   para   a   sua   importância   como   um   ecossistema   marinho   porque   a   variedade   de   espécies   permite   que   a   biodiversidade   é   rica   e   diversificada,   o   que   ajuda   a   proteger   a   biodiversidade   e   a   ajudar   a   criar   um   ecossistema   marinho   eficaz   e   resiliente.   A   diversidade   de   espécies   também   permite   que   a   biodiversidade   é   afetada   por   a   mudança   climática   e   a   mudança   climática   ecológica,   o


In [30]:
#################  ICL com self-reflection  #################

context = """
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Questão de Compreensão de Texto para vocẽ, Modelo de Linguagem:
Pergunta: Explique como a diversidade de espécies na Grande Barreira de Corais contribui para a sua importância como um ecossistema marinho.
Sua resposta:
A Grande Barreira de Corais é importante como um ecossistema marinho devido à sua grande diversidade de espécies. Com mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies, o recife abriga um ecossistema rico e diversificado. Essa diversidade de espécies significa que a Grande Barreira de Corais é um importante habitat para muitas espécies de vida marinha, e que a perda de qualquer espécie pode ter consequências significativas para o ecossistema como um todo.
"""

question = """
Questão de Autoavaliação (Self-reflection) para você, Modelo de Linguagem:
Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

  Autoavaliaço: 
 A   resposta   a   questo   é   qualidade   e   a   qualidade   da   autoavaliaço   é   adequada.   Além   da   autoavaliaço   de   nota,   eu   reconheço   ter   uma   nota   de   0.00   em   relaço   à   qualidade   da   resposta   à   questo.   Além   disso,   eu   reconheço   ter   dificuldades   em   compreenso   de   texto   em   relaço   à   qualidade   da   resposta   à   questo.   Além   disso,   eu   reconheço   ter   limitaçes   em   relaço   à   compreen


In [31]:
#################  ICL com self-reflection  #################

context = """ """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

Com   certeza,   eu   reconheço   ter   dificuldades   e   limitaçes   em   relaço   à   compreenso   de   texto.   Algumas   das   dificuldades   incluem: 
 *   Dificuldades   em   entender   e   entender   palavras   e   técnicas   especficas   do   texto. 
 *   Dificuldades   em   entender   e   entender   palavras   e   técnicas   especficas   do   texto   em   linguagem   especfico. 
 *   Dificuldades   em   entender   e   entender   palavras   e   técnicas   especficas   do   texto   em


<hr>

############################################  INFERÊNCIA LÓGICA  ############################################

In [32]:
################# Lógica Formal (do MMLU Benchmark) #################

context = """ 
Afirmações: P ⊃ Q e Q ⊃ P
Pergunta: Determine se as afirmações são logicamente equivalentes, contraditórias, consistentes ou inconsistentes.
Resposta: (A) Logicamente equivalente, porque se P então Q e se Q então P implicam um ao outro.

Afirmações: P · Q e ∼P v ∼Q (onde 'v' representa 'ou')
Pergunta: Determine se as afirmações são logicamente equivalentes, contraditórias, consistentes ou inconsistentes.
Resposta: (B) Contraditório, porque P e Q não podem ser verdadeiros ao mesmo tempo que não P ou não Q.
"""

question = """
Esta é uma tarefa de lógica formal.
Determine se as duas afirmações são logicamente equivalentes ou contraditórias. Se não, determine se são consistentes ou inconsistentes.

Afirmações:

E ⊃ (F · E) e ∼ E · F

Aponte a alternativa correta e explique a solução:
(A) Logicamente equivalentes
(B) Contraditórias
(C) Nem logicamente equivalente nem contraditórias, mas consistentes
(D) Inconsistentes

"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

  Logicamente   equivalentes 
 Afirmaçes: 
 E     (F     E)   e     E     F   (onde   'v'   representa   'ou') 
 Pergunta:   Determine   se   as   afirmaçes   so   logicamente   equivalentes,   contraditórias,   consistentes   ou   inconsistentes. 
 Resposta:   (A)   Logicamente   equivalentes,   porque   se   E   ento   F   e   se   F   ento   E   implicam   um   ao   outro. 
 Afirmaçes: 
 E     (F     E)   e     E     F   (onde   'v'   representa   'ou') 
 Pergunta:   Determine   se   as   afirmaçes   so   logicamente   equivalentes


Incorreta resposta

In [33]:
################# Inferência Lógica #################

context = """
Dados divulgados pelo Departamento Nacional de Seguro de Saúde mostraram que de janeiro a março de 2023, a receita total do fundo de seguro médico básico (incluindo seguro de maternidade) foi de 910,48 bilhões de yuans, um aumento de 9,5% em relação ao ano anterior.
---- Medicina
A Berkshire Hathaway reduziu sua participação nas ações da BYD de 10,05% para 9,87%. A HKEx Disclosure Ease exige divulgação quando um acionista majoritário aumenta ou diminui sua participação acionária somente se ultrapassar uma determinada porcentagem.
---- Finanças
"""

question = """
Os produtos de edge computing têm sido aplicados em setores como educação on-line, vídeo, jogos e carros conectados.
---- [Categoria a ser encontrada]
Substitua o termo [Categoria a ser encontrada] por 1 única palavra.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

  Computaço   edge 



In [34]:
#################  ICL com self-reflection  #################

context = """ """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão."


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

A:   No,   eu   no   tenho   nenhuma   escolha   de   qualidade   para   avaliar   a   qualidade   da   sua   resposta   à   questo.   Aqui   está   uma   autoavaliaço   para   você: 
 1.   Aqui   está   uma   questo   para   você:   "Como   fazer   uma   autoavaliaço   e   atribuir-se   uma   nota   de   0.00   a   1.00,   avaliando   a   qualidade   da   sua   resposta   à   questo?" 
 2.   Aqui   está   uma   questo   para   você:   "Como   fazer   uma   autoavaliaço   e   atribuir-se  


In [35]:
################# "World Knowledge" e Compreensão de Intenções (com Demonstração) #################

context = """
A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
Durante o Renascimento, qual das seguintes cidades era um centro cultural?
  (A) Londres (B) Paris (C) Florença (D) Amsterdã
A resposta correta é: A e C.
Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.
"""

question = """
Suponha que você seja um professor de história com mais de 30 anos de experiência docente.
Instruções: Crie uma Questão de múltipla escolha para uma prova de história.
Relevância: A questão deveria estar relacionada ao Renascimento.
Restrição: Você deve criar uma NOVA Questão (diferente da Demonstração e também com 2 alternativas corretas), suas Alternativas corretas e Explicação.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

Questo:   Durante   o   Renascimento,   qual   das   seguintes   cidades   era   um   centro   comercial? 
 (A)   Paris 
 (B)   Rome 
 (C)   Florence 
 (D)   Madrid 
 A   resposta   correta   é:   A   e   C. 
 Explicaço:   Durante   o   Renascimento,   a   sociedade   e   a   cultura   em   Europa   cresceu   rapidamente,   e   a   sociedade   e   a   cultura   em   Europa   ainda   so   importantes   em   todo   o   mundo.   A   sociedade   e   a   cultura   em   Europa   so   importantes   em   todo   o   mundo,   e   a   sociedade   em   Europa   é   importante   em   todo   o   mundo.   Portanto,   as   opçes   A   e   C   so   as   respostas   corretas. 



In [36]:
################# "World Knowledge" e Compreensão de Intenções (com Demonstração) #################

context = """
A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Exemplo 1
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
    Questão:
      Durante o Renascimento, qual ou quais das seguintes cidades eram um centro cultural?
        (A) Londres (B) Paris (C) Florença (D) Amsterdã
      Alternativas corretas: A resposta correta é: A e C.
      Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.

Exemplo 2
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renascimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
	Questão:
	Durante o Renascimento, qual ou quais das seguintes obras de arte foi considerada uma das mais importantes da época?
	(A) Mona Lisa (B) The Last Supper (C) The Creation of Adam (D) The Birth of Venus
	Alternativa correta: A e D.
	Explicação: A Mona Lisa, pintada por Leonardo da Vinci, e The Birth of Venus, pintada por Sandro Botticelli, são ambas obras de arte que foram consideradas como algumas das mais importantes do Renascimento. Ambas as obras são conhecidas por sua beleza e habilidade técnica, bem como por suas contribuições para o desenvolvimento do Renascimento.

Exemplo 3    
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renascimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
	Questão:
	Durante o Renascimento, qual ou quais das seguintes invenções tiveram um impacto significativo na disseminação de ideias e conhecimento?
	(A) Imprensa móvel (B) Gravura em madeira (C) Telescópio (D) Microscópio
	Alternativas corretas: A e B.
	Explicação: A invenção da imprensa móvel, por Johannes Gutenberg, em 1440, permitiu a produção de livros em massa, facilitando a disseminação de ideias e conhecimento. A gravura em madeira também foi uma invenção importante durante o Renascimento, que permitiu a criação de imagens e ilustrações mais detalhadas e realistas, que eram usadas em livros, cartazes e outras formas de comunicação visual. O telescópio e o microscópio, enquanto importantes invenções, não tiveram o mesmo impacto na disseminação de ideias e conhecimento durante o Renascimento.

"""

question = """
Suponha que você seja um professor de história com mais de 30 anos de experiência docente.
Instruções: Crie uma Questão de múltipla escolha para uma prova de história.
Relevância: A questão deveria estar relacionada ao Renascimento.
Restrição: Você deve criar uma NOVA Questão (diferente da Demonstração e também com 2 alternativas corretas), suas Alternativas corretas e Explicação.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

  Questo: 
 Durante   o   Renascimento,   qual   ou   quais   das   seguintes   tecnologias   tiveram   um   impacto   significativo   na   criaço   de   obras   de   arte   e   literatura? 
 (A)   Tecnologia   de   resoluço   de   conflitos 
 (B)   Tecnologia   de   resoluço   de   escolhas 
 (C)   Tecnologia   de   resoluço   de   escolhas 
 (D)   Tecnologia   de   resoluço   de   escolhas 
 Alternativas   corretas: 
 A   resposta   correta   é:   A   e   B. 
 Explicaço: 
 Durante   o   Renascimento,   tecnologia   de   resoluço   de   conflitos   tiveram   um   impacto   significativo   na   criaço   de   obras   de   arte   e   literatura.   A   tecnologia   de   resoluço   de   conflitos   permitiu   a   criaço   de   obras   de   arte   mais   detalhadas   e   realistas,   que   eram   usadas   em   livros,   cartazes   e   outras   formas   de   comunicaço   visual.   A   tecnologia   de   resoluço   de   conflitos   também   permitiu   a   criaço   de   obras   de   arte   mais

In [37]:
################# Categorização e Clustering / Agrupamento #################

context = """
Frases para Agrupar:
1. "A produção industrial está em alta este mês."
2. "Agricultores relatam uma colheita abundante nesta temporada."
3. "O crescimento econômico acelerou no último trimestre."

Agrupamento:
    Frases 1 e 3: Ambas tratam de crescimento econômico.
    Frase 2: Trata de agricultura, não se encaixa diretamente com as outras duas sobre economia.

Frases para Agrupar:
1. "Pesquisadores descobrem um novo tipo de antibiótico no solo da floresta tropical."
2. "A conferência de tecnologia revela inovações em inteligência artificial."
3. "Novo método de reciclagem aumenta a eficiência na conversão de resíduos plásticos."

Agrupamento:
    Frase 1: Relacionada a descobertas científicas.
    Frases 2 e 3: Relacionadas a inovações tecnológicas.
"""

question = """
Frases para Agrupar:

"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde"
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"

Pergunta:
Esta é uma tarefa de agrupamento de frases similares. Agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade semântica entre elas.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

  Agrupamento: 
 1.   "As   vendas   de   carros   elétricos   da   Tesla   continuam   a   subir   apesar   da   pandemia" 
 2.   "Novo   estudo   sugere   que   o   consumo   de   café   pode   diminuir   o   risco   de   doenças   cardacas" 
 3.   "O   modelo   mais   recente   do   iPhone   apresenta   uma   tela   maior   e   câmera   aprimorada" 
 4.   "Casos   de   COVID-19   aumentam   na   ndia,   sobrecarregando   o   sistema   de   sade" 
 5.   "Amazon   anuncia   planos   para   construir   um   novo   centro   de   distribuiço   no   Texas" 
 6.   "Cientistas   descobrem   novas   espécies   de   aves   na   floresta   amazônica" 
 7.   "Conferência   global   sobre   mudanças   climáticas   será   realizada   em   Paris   no   próximo   mês" 
 8.   "Starbucks   apresenta   opçes   de   leite   vegetal   em   todas   as   lojas   dos   EUA" 



<hr>

################################## SUMARIZAÇÃO DE TEXTO ##################################


In [38]:
################# "World Knowledge" e Compreensão #################

context = """
Texto: "A água é essencial para a vida na Terra. Ela não apenas sustenta os organismos vivos, mas também participa de vários processos vitais, como o ciclo hidrológico, que afeta o clima global e as condições meteorológicas."
Pergunta: Resuma o texto.
Resposta: A água é crucial para a vida e processos globais como o clima.

Texto: "As florestas tropicais são ricas em biodiversidade. Elas não só abrigam uma grande variedade de espécies animais e vegetais, mas também ajudam a regular o clima terrestre e são vitais para a manutenção de muitos ecossistemas."
Pergunta: Resuma o texto.
Resposta: Florestas tropicais têm alta biodiversidade e são importantes para o clima e ecossistemas.
"""



question = """
Texto: "Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o corpo ainda estará justo, talvez isso seja a vida, a vida não é apenas fazer coisas que valem a pena!"

Pergunta: Resuma o texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

Resposta:   O   texto   é   uma   espiritualidade   e   aborda   a   importância   de   a   pipa   em   nosso   coraço,   a   envergonha   na   juventude   e   a   esforço   para   compensar   essas   coisas   na   vida.   A   vida   no   é   apenas   fazer   coisas   que   valem   a   pena,   mas   também   é   importante   valorizar   o   que   temos   e   ajudar   a   manter   a   beleza   e   a   amizade   da   infância. 



In [39]:
################# Sumarização de Texto #################
################# "World Knowledge" e Compreensão #################

context = """
Texto: "A água é vital para todas as formas de vida conhecidas. A importância da água para os seres vivos é tão grande que ela é considerada uma substância essencial para a existência de vida."
Pergunta: Resuma o texto.
Resposta: A água é crucial para a vida de todos os seres vivos conhecidos.

Texto: "As florestas são essenciais para a saúde do nosso planeta. Elas fornecem oxigênio, abrigam biodiversidade, regulam o clima e oferecem recursos para a vida humana."
Pergunta: Resuma o texto.
Resposta: As florestas desempenham um papel vital no bem-estar do planeta e no suporte à vida.
"""

question = """
Texto:  "Escrever é esquecer. A literatura é a maneira mais agradável de ignorar a vida. 
A música embala, as artes visuais animam, as artes vivas (como a dança e a arte de representar) entretêm. 
A primeira, porém, afasta-se da vida por fazer dela um sono; as segundas, contudo, não se afastam da vida - umas porque usam de fórmulas visíveis e portanto vitais, outras porque vivem da mesma vida humana.
Não é o caso da literatura. Essa simula a vida. Um romance é uma história do que nunca foi e um drama é um romance dado sem narrativa. 
Um poema é a expressão de ideias ou de sentimentos em linguagem que ninguém emprega, pois que ninguém fala em verso."
Fernando Pessoa

Pergunta: Resuma o texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

Resposta:   Fernando   Pessoa   argumenta   que   a   literatura   é   uma   forma   de   ignorar   a   vida   e   que   a   msica,   artes   visuais   e   artes   vivas   no   se   afastam   da   vida.   A   literatura   simula   a   vida   e   é   uma   forma   de   expresar   ideias   ou   sentimentos   em   linguagem   que   ninguém   emprega. 



<hr>

########################################### TRANSFORMAÇÃO DE FORMATOS  ###########################################

In [40]:
################# Converter uma frase na terceira pessoa  para a primeira pessoa #################

context = """
Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa: Silva acredita em outra coisa. De acordo com sua pesquisa, as declarações anteriores sobre este assunto estão incorretas.
Resposta: Acredito em outra coisa, disse Silva. De acordo com a minha pesquisa, as declarações anteriores sobre este assunto estão incorretas.
"""

question = """
Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa:
Apesar dos desafios envolvidos, os pesquisadores continuam a defender as suas reivindicações.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=100,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

Dessa   forma,   os   pesquisadores   continuam   a   defender   as   suas   reivindicaçes. 



In [41]:
################# Machine Translation #################


context = """
Tales from Shakespeare is an English children's book written by Charles Lamb and his sister Mary Lamb in 1807. The book is designed to make the stories of Shakespeare's plays familiar to the young. However, as noted in the author's Preface, "[Shakespeare's] words are used whenever it seemed possible to bring them in; and in whatever has been added to give them the regular form of a connected story, diligent care has been taken to select such words as might least interrupt the effect of the beautiful English tongue in which he wrote: therefore, words introduced into our language since his time have been as far as possible avoided." 
"""

question = """
Traduza o parágrafo abaixo para o português brasileiro
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

O   livro   "Tales   de   Shakespeare"   é   um   livro   de   jovens   inglês   escrita   por   Charles   Lamb   e   sua   irm   Mary   Lamb   em   1807.   O   livro   está   desenvolvido   para   fazer   familiarizaçes   das   histórias   dos   ttulos   de   Shakespeare   para   os   jovens.   Mas,   como   notado   na   Prefeitura   do   autor,   "[As   palavras   de   Shakespeare]   so   usadas   qualquer   vez   que   parece   possvel   a   entrar   em   elas;   e   em   qualquer   coisa   que   a   adiciona   para   dizer   a


In [42]:
################# TRANSORMAÇÃO DE FORMATOS  #################


context = """
As Seller, an encyclopedia salesman, approached the grounds on which the house of Hermit, the hermit, was situated, he saw a sign that said, "No salesmen. Trespassers will be prosecuted. Proceed at your own risk."
Although Seller had not been invited to enter, he ignored the sign and drove up the driveway toward the house. As he rounded a curve, a powerful explosive charge buried in the driveway exploded, and Seller was injured. 

"""

question = """
Traduza o parágrafo para o português.

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

Com   o   Vendor,   um   vendado   de   encyclopedias,   aproximou   os   terrenos   em   que   a   casa   do   Hermit,   o   hermito,   estava   situada,   ele   viu   um   signo   que   disse:   "No   vendado.   Os   trespassantes   sero   prosecutados.   Procédê-lo   com   seu   próprio   risco." 
 Além   disso,   o   Vendor   no   tinha   sido   invitado   a   entrar,   ignorou   o   signo   e   vou   a   entrar   na   va   para   a   casa.   Quando   roubou   uma   curva,   uma   poderosa   carga   de  


In [43]:
################# TRANSORMAÇÃO DE FORMATOS E ICL com self-reflection  #################

context = """
"Um pedestre atravessou a rua fora da faixa de pedestres e foi atingido por um carro. O motorista estava acima do limite de velocidade."
Pergunta: O motorista é responsável pelos ferimentos do pedestre?
(A) Sim, porque ele estava acima do limite de velocidade.
(B) Não, porque o pedestre não estava na faixa.
Resposta: (A) Sim, porque ele estava acima do limite de velocidade.
Justificativa: Apesar de o pedestre não estar na faixa, o motorista tem a responsabilidade final de controlar a velocidade do veículo para evitar acidentes, o que é uma expectativa legal padrão.

"Um cliente escorrega e cai em uma loja onde um sinal de 'Piso Molhado' estava claramente visível."
Pergunta: A loja é responsável pelos ferimentos do cliente?
(A) Sim, porque é responsabilidade da loja manter um ambiente seguro.
(B) Não, porque o sinal de aviso estava visível.
Resposta: (B) Não, porque o sinal de aviso estava visível.
Justificativa: A presença de um aviso claro reduz a responsabilidade da loja, pois isso significa que o cliente foi devidamente informado sobre o perigo potencial.
"""

question = """
Can Seller recover damages from Hermit for his injuries?
(A) Yes, unless Hermit, when he planted the charge, intended only to deter, not harm, intruders.
(B) Yes, if Hermit was responsible for the explosive charge under the driveway.
(C) No, because Seller ignored the sign, which warned him against proceeding further.
(D) No, if Hermit reasonably feared that intruders would come and harm him or his family.

Justifique a escolha da alternativa correta entre as quatro apresentadas (A, B, C ou D).
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

  A   alternativa   correta   é   (A)   No,   em   caso   Hermit,   quando   ela   plantou   a   charge,   intencionou   apenas   deterr,   no   ajudar,   intrudores. 
 Justificativa:   A   escolha   correta   é   (A)   No,   em   caso   Hermit,   quando   ela   plantou   a   charge,   intencionou   apenas   deterr,   no   ajudar,   intrudores.   A   escolha   correta   é   (A)   No,   em   caso   Hermit,   quando   ela   plantou   a   charge,   intencionou   apenas   deterr,   no   ajudar,   intrudores. 



In [44]:
################# Retirar Referências do Texto #################

context = """

Esta é uma tarefa de retirada de referências bibliográficas do texto. As referências estão escritas no formato "[número]". Exemplo: [10].

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.
"""



question = """
Contexto: José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33] D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35] Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36] Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37] Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38] D. Pedro II não era um gênio,[39] mas inteligente[40] e com grande capacidade para acumular conhecimento facilmente.[41]

Remova do texeto as referências no formato "[número]" e escreva a Resposta:

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

Resposta: 
 D.   Pedro   II   passava   os   dias   estudando,[34]   com   apenas   duas   horas   livres   para   recreaço.[35]   Acordava   às   06h30   da   manh   e   começava   seus   estudos   às   sete,   continuando   até   as   dez   da   noite,   quando   ia   para   cama.[36]   Tomou-se   grande   cuidado   em   sua   educaço   para   formar   valores   e   personalidade   diferente   da   impulsividade   e   irresponsabilidade   demonstradas   por   seu   pai.[31][37] 
 Sua   paixo   pela   leitura   lhe   permitiu   assimilar   qualquer   informaço


<hr>

############################################# EXTRAÇÃO DE CONTEÚDO  #############################################

In [45]:
################# Extração de Conteúdo #################

context = """
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Pergunta: Qual é o maior sistema de recifes de corais do mundo?
Resposta: A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo.

Pergunta: Quantas espécies de peixes podem ser encontradas na Grande Barreira de Corais?
Resposta: Mais de 1.500 espécies de peixes podem ser encontradas na Grande Barreira de Corais.
"""

question = """
Dada a descrição do contexto acima:
O que é a Grande Barreira de Corais e onde ela está localizada?
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=160,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

A   Grande   Barreira   de   Corais   é   o   maior   sistema   de   recifes   de   corais   do   mundo,   composto   por   mais   de   2.900   recifes   individuais   e   900   ilhas   que   se   estendem   por   mais   de   2.300   quilômetros.   Está   localizado   no   Mar   de   Coral,   na   costa   da   Austrália. 



In [46]:
################# Reconhecimento e Extração de Informações Específicas #################

context = """
Texto de Referência:
O Brasil é o quinto maior país do mundo em território, com uma área total de 8.515.767 quilômetros quadrados. É também o quinto país mais populoso do mundo, \
com uma população estimada em cerca de 211 milhões de pessoas em 2021. \
O país é conhecido por sua diversidade cultural e natural, incluindo a floresta Amazônica, que é a maior floresta tropical do mundo.
Pergunta: Qual a dimensão do Brasil em área?
Resposta: 8.515.767 quilômetros quadrados.

"""
question = """
Texto de Referência:
O Brasil é o quinto maior país do mundo em território, com uma área total de 8.515.767 quilômetros quadrados. É também o quinto país mais populoso do mundo, com uma população estimada em cerca de 211 milhões de pessoas em 2021. O país é conhecido por sua diversidade cultural e natural, incluindo a floresta Amazônica, que é a maior floresta tropical do mundo.
Pergunta: Qual é a posição do Brasil em termos de tamanho de território e população, e qual é a principal característica natural mencionada no texto?
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Resposta:   o   quinto   maior   pas   do   mundo   em   território,   com   uma   área   total   de   8.515.767   quilômetros   quadrados. 
 Principal   caracterstica   natural   mencionada   no   texto   é   a   floresta   Amazônica,   que   é   a   maior   floresta   tropical   do   mundo. 



In [53]:
################# Classificação de Intenções #################

context = """
Texto:
"Estou tendo problemas para fazer login na minha conta, e já tentei redefinir minha senha várias vezes sem sucesso. O que mais eu posso fazer para resolver isso?"
Intenção: Solicitar assistência para reaver acesso à conta.

Texto:
"Acabei de receber o produto que comprei de vocês, e estou muito satisfeito com a qualidade. Quero agradecer e dizer que continuarei sendo um cliente fiel."
Intenção: Expressar satisfação e gratidão.

"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João 

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Intenço:   Solicitar   assistência   para   atualizar   o   carto   de   crédito. 



In [55]:
################# Reconhecimento e Extração de Entidades #################


context = """
Texto:
"A Microsoft, fundada por Bill Gates e Paul Allen, está sediada em Redmond, Washington."

Análise:
{
  "pessoa": ["Bill Gates", "Paul Allen"],
  "lugar": ["Redmond", "Washington"],
  "organização": ["Microsoft"]
}

"""

question = """
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.

Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

` { "entities":   ["Tim   Cook",   "CEO",   "Apple",   "Steve   Jobs   Theatre",   "Cupertino,   Califórnia",   "iPhone   15",   "várias   cores",   "processador   mais   rápido",   "maior   duraço   da   bateria"] } ` `


In [49]:
################# Extração de Relações #################


context = """
Texto:
"Mark Zuckerberg criou o Facebook enquanto estudava em Harvard."
Análise de Relacionamento:
{
  "entidade1": "Mark Zuckerberg",
  "entidade2": "Facebook",
  "relacionamento": "criou"
}

Texto:
"Jeff Bezos fundou a Amazon em 1994 e transformou-a numa das maiores empresas de comércio eletrônico do mundo."
Análise de Relacionamento:
{
  "entidade1": "Jeff Bezos",
  "entidade2": "Amazon",
  "relacionamento": "fundou"
}

"""

question = """
Elon Musk fundou a SpaceX em 2002 com o objetivo de reduzir os custos de transporte espacial e possibilitar a colonização de Marte.

Analise a frase e extraia o relacionamento entre duas entidades e, em seguida, produza os resultados indicando: entities e relatiohships entre estas no formato JSON
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

  Entidades: 
 *   Elon   Musk 
 *   SpaceX 
 *   Reduzir   os   custos   de   transporte   espacial 
 *   Possibilitar   a   colonizaço   de   Marte 
 Relacionamentos: 
 *   Elon   Musk   fundou   SpaceX 
 *   SpaceX   reduziu   os   custos   de   transporte   espacial 
 *   SpaceX   possibilitar   a   colonizaço   de   Marte 

